# Galactic models comparison - short version

In [ ]:
from healpy.newvisufunc import projview
from healpy.rotator import Rotator
import healpy as hp
import traceback

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

import plotly.graph_objects as go
import plotly.express as px

from sklearn.neighbors import KernelDensity

import os
import sys
from radiocalibrationtoolkit import *

In [ ]:
# some global plot settings
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"
plt.rcParams['font.size'] = 16
plt.rcParams['legend.fontsize']= 14

plt.rcParams['xtick.major.width']= 2
plt.rcParams['ytick.major.width']= 2

plt.rcParams['xtick.major.size']= 5
plt.rcParams['ytick.major.size']= 5

plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14

In [ ]:
# Prepare instances
lfmap = LFmap()
lfss = LowFrequencySkyModel(freq_unit="MHz")
gsm2008 = GlobalSkyModel(freq_unit="MHz")
gsm2016 = GlobalSkyModel2016(freq_unit="MHz")
haslam = HaslamSkyModel(freq_unit="MHz", spectral_index=-2.53)
ssm = SSM()
gmoss = GMOSS()
ulsa_fdi = ULSA(index_type='freq_dependent_index')
ulsa_ci = ULSA(index_type='constant_index')
ulsa_dpi = ULSA(index_type='direction_dependent_index')

In [ ]:
frequency_MHz = 45

lfmap_map = lfmap.generate(frequency_MHz)
lfss_map = lfss.generate(frequency_MHz)
gsm2008_map = gsm2008.generate(frequency_MHz)
gsm2016_map = gsm2016.generate(frequency_MHz)
haslam_map = haslam.generate(frequency_MHz)
ssm_map = ssm.generate(frequency_MHz)
gmoss_map = gmoss.generate(frequency_MHz)
ulsa_fdi_map = ulsa_fdi.generate(frequency_MHz)
ulsa_ci_map = ulsa_ci.generate(frequency_MHz)
ulsa_dpi_map = ulsa_dpi.generate(frequency_MHz)

In [ ]:
# Check size of the arrays and NSIDE number and angular resolution

map_list = [lfmap_map, lfss_map, gsm2008_map, gsm2016_map, haslam_map, ssm_map, gmoss_map, ulsa_fdi_map, ulsa_ci_map, ulsa_dpi_map]
[print_map_properties(m) for m in map_list]

In [ ]:
# convert to same NSIDE
new_nside = 64
lfmap_map_N = hp.ma(hp.pixelfunc.ud_grade(lfmap_map, new_nside))
lfss_map_N = hp.ma(hp.pixelfunc.ud_grade(lfss_map, new_nside))
gsm2008_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2008_map, new_nside))
gsm2016_map_N = hp.ma(hp.pixelfunc.ud_grade(gsm2016_map, new_nside))
ssm_map_N = hp.ma(hp.pixelfunc.ud_grade(ssm_map, new_nside))
haslam_map_N = hp.ma(hp.pixelfunc.ud_grade(haslam_map, new_nside))
gmoss_map_N = hp.ma(hp.pixelfunc.ud_grade(gmoss_map, new_nside))
ulsa_fdi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_fdi_map, new_nside))
ulsa_ci_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_ci_map, new_nside))
ulsa_dpi_map_N = hp.ma(hp.pixelfunc.ud_grade(ulsa_dpi_map, new_nside))

In [ ]:
map_dict = {
    "LFSS": lfss_map_N,
    "GSM08": gsm2008_map_N,
    "GSM16": gsm2016_map_N,
    "Haslam": haslam_map_N,
    "LFmap": lfmap_map_N,
    "SSM": ssm_map_N,
    "GMOSS": gmoss_map_N,
    "ULSA": ulsa_fdi_map_N,
    # "ULSA2": ulsa_ci_map_N,
    # "ULSA3": ulsa_dpi_map_N,
}

ulsa_map_dict = {
    "ULSA FDI": ulsa_fdi_map_N,
    "ULSA CI": ulsa_ci_map_N,
    "ULSA DPI": ulsa_dpi_map_N,
}


In [ ]:
# Print the maps in the dictionary
[(key, map_dict[key]) for key in map_dict]

## Galactic coordinates

In [ ]:
# Prints all maps
cmap = "jet"
projection_type = "mollweide"

for key in map_dict:

    projview(
        map_dict[key],
        norm="log",
        coord=["G"],
        graticule=True,
        graticule_labels=True,
        unit="Temperature ln[K]",
        xlabel="RA",
        ylabel="DEC",
        cb_orientation="vertical",
        min=3500,
        max=35000,
        latitude_grid_spacing=30,
        projection_type=projection_type,
        title=key,
        xtick_label_color="white",
        cmap=cmap,
    )

In [ ]:
compare_maps(
    map_dict,
    main_title="Frequency: {} MHz; Compared on {:.2f}{} angular resolution".format(
        frequency_MHz, np.rad2deg(hp.pixelfunc.nside2resol(new_nside)), chr(176)
    ),
)

In [ ]:
compare_maps(
    ulsa_map_dict,
    main_title="Frequency: {} MHz; Compared on {:.2f}{} angular resolution".format(
        frequency_MHz, np.rad2deg(hp.pixelfunc.nside2resol(new_nside)), chr(176)
    ),
)

## Local obsever

In [ ]:
lst = 18
latitude = -35.206667
frequency_MHz = 45
main_title = "Frequency: {} MHz; Compared on {:.2f}{} angular resolution; LST:{}; Latitude:{:.2f}".format(
    frequency_MHz, np.rad2deg(hp.pixelfunc.nside2resol(new_nside)), chr(176), lst, latitude
)

# Local coordinates at LST time "LSTtime" at latitude "latitude"
rotation_parameters = create_rotation_parameters(lst, latitude)

In [ ]:
# Prints all maps
cmap = "jet"
projection_type = "mollweide"

for key in map_dict:

    projview(
        map_dict[key],
        norm="log",
        coord=['G','C'],
        rot=rotation_parameters,
        graticule=True,
        graticule_labels=True,
        unit="Temperature ln[K]",
        xlabel="azimuth",
        ylabel="altitude",
        cb_orientation="vertical",
        min=3500,
        max=35000,
        latitude_grid_spacing=30,
        projection_type=projection_type,
        title=key,
        xtick_label_color="white",
        cmap=cmap,
    )

In [ ]:
# Create mask, at local coordinates we see only a hemisphere
mask = create_local_mask(new_nside, rotation_parameters)

In [ ]:
# Test
m = map_dict['LFmap'].copy()
m.mask = mask

projview(
    m,
    norm="log",
    coord=["G",'C'],
    rot=rotation_parameters,
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="azimuth",
    ylabel="altitude",
    cb_orientation="vertical",
    min=3500,
    max=35000,
    latitude_grid_spacing=30,
    xtick_label_color="white",
    title=main_title,
    cmap=cmap,
)

projview(
    m,
    norm="log",
    coord=["G"],
    rot=False,
    graticule=True,
    graticule_labels=True,
    unit="Temperature ln[K]",
    xlabel="RA",
    ylabel="DEC",
    cb_orientation="vertical",
    min=3500,
    max=35000,
    latitude_grid_spacing=30,
    xtick_label_color="white",
    title=main_title,
    cmap=cmap,
)

### In local coordinates

In [16]:
compare_maps(
    map_dict, rotation_parameters=rotation_parameters, coord=['G','C'], main_title=main_title, mask=mask
)

### In galactic coordinates (masked)

In [ ]:
compare_maps(map_dict, rotation_parameters=False, main_title=main_title, mask=mask)

In [ ]:
lst = 18
latitude = -35.206667
rotation_parameters = create_rotation_parameters(lst, latitude)
mask = create_local_mask(nside, rotation_parameters)
gsm2016_map = hp.ma(gsm2016_map)
gsm2016_map.mask = mask
